In [1]:
import pandas as pd
import os
import re
import numpy as np
import cv2

In [2]:
base_dir = 'D:/KHH/team_project/json_data/'

In [3]:
os.listdir(base_dir)

['Barbell_Dumbbell.csv',
 'Barbell_Dumbbell_validation.csv',
 'Bodyweight.csv',
 'Bodyweight_validation.csv',
 'Equipment.csv',
 'Equipment_validation.csv']

In [4]:
json_df = pd.DataFrame([])

In [5]:
for file_csv in os.listdir(base_dir):
    if 'validation' not in file_csv:
        print(file_csv)
        json_df = pd.concat([json_df, pd.read_csv(base_dir+file_csv)], axis=0)

Barbell_Dumbbell.csv
Bodyweight.csv
Equipment.csv


In [6]:
json_df.reset_index(drop=True, inplace=True)

In [7]:
json_df['cnn_file_name'] = json_df['img_key'].apply(lambda x : x.replace('/', '@'))

In [8]:
json_df.sample()['img_key']

2373717    Day36_201108_F/4/C/629-1-3-29-Z5_C/629-1-3-29-...
Name: img_key, dtype: object

In [9]:
json_df.type.value_counts()

type
바벨/덤벨    1317620
맨몸 운동    1094680
기구        340375
Name: count, dtype: int64

In [10]:
json_df['folder'] = json_df['img_key'].apply(lambda x: "/".join(x.split("/")[:4]))

In [11]:
json_df.folder.value_counts()

folder
Day02_200922_F/4/B/229-2-1-12-Z1_B     21
Day02_200922_F/4/C/229-2-1-12-Z1_C     21
Day02_200922_F/4/E/229-2-1-12-Z1_E     21
Day02_200922_F/4/A/229-1-1-07-Z1_A     21
Day02_200922_F/4/D/229-2-1-12-Z1_D     21
                                       ..
Day11_201008_F/1/E/310-2-1-14-Z36_E     8
Day11_201008_F/1/D/310-2-1-14-Z36_D     8
Day11_201008_F/1/C/310-2-1-14-Z36_C     8
Day11_201008_F/1/B/310-2-1-14-Z36_B     8
Day11_201008_F/1/A/310-2-1-14-Z36_A     8
Name: count, Length: 172200, dtype: int64

In [12]:
def stratified_sampling_and_split(df, category_column, sample_size=10000):
    """
    각 카테고리에서 지정된 개수만큼 층화추출
    """
    sampled_dfs = []
    
    # 각 카테고리별 샘플링
    for category in df[category_column].unique():
        category_df = df[df[category_column] == category]
        sampled_df = category_df.sample(n=sample_size)  # 층화추출
        sampled_dfs.append(sampled_df)
    
    # 샘플링된 데이터프레임 합치기
    final_df = pd.concat(sampled_dfs).reset_index(drop=True)
    return final_df

In [13]:
test_data = stratified_sampling_and_split(json_df, 'type', 10)

In [14]:
test_data = test_data.sample(frac=1)

In [15]:
test_data

,img_key,type,workout,conditions,description,cnn_file_name,folder
9,Day08_201005_F/4/D/441-2-1-20-Z18_D/441-2-1-20...,바벨/덤벨,덤벨 컬,"[{'condition': '팔꿈치 위치 고정', 'value': True}, {'...",1 팔꿈치 고정 2 손목 중립 3 척추 중립 4 이완시 긴장 4 으쓱 없음,Day08_201005_F@4@D@441-2-1-20-Z18_D@441-2-1-20...,Day08_201005_F/4/D/441-2-1-20-Z18_D
0,Day29_201031_F/9/A/283-2-1-14-Z114_A/283-2-1-1...,바벨/덤벨,바벨 데드리프트,"[{'condition': '척추의 중립', 'value': True}, {'con...",무릎 모인상태,Day29_201031_F@9@A@283-2-1-14-Z114_A@283-2-1-1...,Day29_201031_F/9/A/283-2-1-14-Z114_A
14,Day28_201030_F/2/C/136-1-1-05-Z107_C/136-1-1-0...,맨몸 운동,스텝 백워드 다이나믹 런지,"[{'condition': '앞다리 무릎 각도 90도', 'value': True}...",정자세에서 왼쪽보고 몸통 옆으로 갸우뚱,Day28_201030_F@2@C@136-1-1-05-Z107_C@136-1-1-0...,Day28_201030_F/2/C/136-1-1-05-Z107_C
7,Day11_201008_F/1/D/266-2-1-13-Z36_D/266-2-1-13...,바벨/덤벨,덤벨 벤트오버 로우,"[{'condition': '척추의 중립', 'value': True}, {'con...",무릎 반동하면서 몸에서 많이 떨어지고 손목 안쪽으로 말린 상태,Day11_201008_F@1@D@266-2-1-13-Z36_D@266-2-1-13...,Day11_201008_F/1/D/266-2-1-13-Z36_D
28,Day22_201023_F/8/A/711-3-5-34-Z114_A/711-3-5-3...,기구,딥스,"[{'condition': '상체 살짝 숙임 유지', 'value': False},...","상체 들고 고개 들고 팔꿈치 조금만 접고 (몸 일직선, 고개 과도하게 뒤로, 까딱)",Day22_201023_F@8@A@711-3-5-34-Z114_A@711-3-5-3...,Day22_201023_F/8/A/711-3-5-34-Z114_A
2,Day03_200923_F/4/B/349-2-1-16-Z1_B/349-2-1-16-...,바벨/덤벨,바벨 런지,"[{'condition': '앞다리 무릎 각도 90도', 'value': True}...",하늘보고 몸통 상체 뒤로 젖힘,Day03_200923_F@4@B@349-2-1-16-Z1_B@349-2-1-16-...,Day03_200923_F/4/B/349-2-1-16-Z1_B
22,Day21_201022_F/2/D/740-3-5-37-Z100_D/740-3-5-3...,기구,랫풀 다운,"[{'condition': '수축 시 적당한 상체 젖힘', 'value': True...",축 매달리고,Day21_201022_F@2@D@740-3-5-37-Z100_D@740-3-5-3...,Day21_201022_F/2/D/740-3-5-37-Z100_D
1,Day11_201008_F/4/C/346-2-1-16-Z37_C/346-2-1-16...,바벨/덤벨,바벨 런지,"[{'condition': '앞다리 무릎 각도 90도', 'value': True}...",발방향 안쪽 몸통 옆으로 갸우뚱 몸통 상체 뒤로 젖힘,Day11_201008_F@4@C@346-2-1-16-Z37_C@346-2-1-16...,Day11_201008_F/4/C/346-2-1-16-Z37_C
27,Day20_201021_F/4/B/755-3-5-38-Z94_B/755-3-5-38...,기구,페이스 풀,"[{'condition': '수축 시 팔꿈치 각도 90도', 'value': Tru...",손 몸쪽으로,Day20_201021_F@4@B@755-3-5-38-Z94_B@755-3-5-38...,Day20_201021_F/4/B/755-3-5-38-Z94_B
6,Day04_200924_F/4/D/420-2-1-19-Z1_D/420-2-1-19-...,바벨/덤벨,바벨 컬,"[{'condition': '팔꿈치 위치 고정', 'value': True}, {'...",손목 꺾고 툭 내려놓기,Day04_200924_F@4@D@420-2-1-19-Z1_D@420-2-1-19-...,Day04_200924_F/4/D/420-2-1-19-Z1_D


In [16]:

# 운동자세중 준비자세를 제외한 이미지만 추출
def load_images(image_paths, categories, base_dirs):
    """
    이미지 경로 리스트와 카테고리를 받아 OpenCV로 로드 후 넘파이 배열로 변환
    """
    image_data = []
    
    # 카테고리별 폴더 매핑
    category_mapping = {
        '바벨/덤벨': 'Barbell_Dumbbell/',
        '맨몸 운동': 'Bodyweight/',
        '기구': 'Equipment/'
    }
    
    # 파일명에서 숫자를 추출하는 정규식 패턴
    pattern = r"-0*(\d+)\.jpg$"

    non_img_idx = []  # 로드 실패한 이미지 인덱스 리스트
    idx = 0  # 현재 인덱스
    
    for img_path, category in zip(image_paths, categories):
        category_dir = category_mapping.get(category, '')
        
        # 정규 표현식으로 파일명에서 숫자 추출
        if (match := re.search(pattern, img_path)):
            file_num = int(match.group(1))  # 정수 변환
            
            # 숫자가 5 ~ 25 범위 안에 있는 경우만 로드
            if 5 <= file_num <= 25:
                full_img_path = os.path.join(base_dirs, category_dir, img_path)  # 경로 결합
                img = cv2.imread(full_img_path)

                if img is not None:
                    image_data.append(img)
                    print(full_img_path, '로드 완료')
                else:
                    print(f"Error loading: {full_img_path}")
                    non_img_idx.append(idx)
        
        idx += 1  # 인덱스 증가
    
    return np.array(image_data, dtype=object), non_img_idx

In [27]:
import os
import cv2
import numpy as np

def load_images(folder_image_paths, categories, base_dirs, img_size=(256, 256)):
    """
    특정 파일 그룹 (folder_image_path 기준)으로 이미지를 묶어 5D NumPy 배열로 변환
    (batch_size, num_images_per_group, height, width, channels)
    """
    
    category_mapping = {
        '바벨/덤벨': 'Barbell_Dumbbell/',
        '맨몸 운동': 'Bodyweight/',
        '기구': 'Equipment/'
    }
    
    image_groups = []  # 최종 이미지 데이터 저장 (5D 텐서)
    non_img_idx = []  # 로드 실패한 이미지 인덱스 저장
    
    idx = 0  # 현재 인덱스
    max_images_per_group = 0  # 최대 이미지 개수 찾기
    
    all_groups = []  # 모든 그룹을 저장할 리스트
    
    for folder_image_path, category in zip(folder_image_paths, categories):
        category_dir = category_mapping.get(category, '')
        folder_image_path = folder_image_path.replace("/", "@")  # 파일명 변환

        # 해당 카테고리 디렉토리에서 파일 리스트 가져오기
        img_list = sorted(os.listdir(base_dirs + category_dir))  # 파일 정렬 (순서 유지)
        
        # folder_image_path와 일치하는 파일 찾기
        img_paths = [img for img in img_list if folder_image_path in img]

        # 이미지 로드 및 전처리
        image_group = []
        for img_path in img_paths:
            img = cv2.imread(os.path.join(base_dirs, category_dir, img_path))
            if img is not None:
                image_group.append(img)
                print(img_path, '로드 완료')
            else:
                print(f"Error loading: {img_path}")
                non_img_idx.append(idx)

        if image_group:
            max_images_per_group = max(max_images_per_group, len(image_group))  # 최대 이미지 개수 업데이트
            all_groups.append(image_group)  # 그룹 저장
        
        idx += 1  # 인덱스 증가
    
    # 모든 그룹을 같은 개수로 맞추기 (Zero Padding 적용)
    for group in all_groups:
        while len(group) < max_images_per_group:
            group.append(np.zeros((img_size[0], img_size[1], 3), dtype=np.float32))  # Zero Padding 추가
        
        image_groups.append(np.array(group, dtype=np.float32) / 255.0)  # 정규화 후 NumPy 변환
    
    # 최종적으로 (batch_size, num_images_per_group, height, width, channels) 형태로 변환
    image_groups = np.array(image_groups, dtype=np.float32)  

    return image_groups, non_img_idx


In [28]:
img_base_dir = 'D:/KHH/team_project/img/'

In [29]:
train_X, faild_idx = load_images(test_data['folder'], test_data['type'], img_base_dir)

Day08_201005_F@4@D@441-2-1-20-Z18_D@441-2-1-20-Z18_D-0000001.jpg 로드 완료
Day08_201005_F@4@D@441-2-1-20-Z18_D@441-2-1-20-Z18_D-0000003.jpg 로드 완료
Day08_201005_F@4@D@441-2-1-20-Z18_D@441-2-1-20-Z18_D-0000005.jpg 로드 완료
Day08_201005_F@4@D@441-2-1-20-Z18_D@441-2-1-20-Z18_D-0000007.jpg 로드 완료
Day08_201005_F@4@D@441-2-1-20-Z18_D@441-2-1-20-Z18_D-0000009.jpg 로드 완료
Day08_201005_F@4@D@441-2-1-20-Z18_D@441-2-1-20-Z18_D-0000011.jpg 로드 완료
Day08_201005_F@4@D@441-2-1-20-Z18_D@441-2-1-20-Z18_D-0000013.jpg 로드 완료
Day08_201005_F@4@D@441-2-1-20-Z18_D@441-2-1-20-Z18_D-0000015.jpg 로드 완료
Day08_201005_F@4@D@441-2-1-20-Z18_D@441-2-1-20-Z18_D-0000017.jpg 로드 완료
Day08_201005_F@4@D@441-2-1-20-Z18_D@441-2-1-20-Z18_D-0000019.jpg 로드 완료
Day08_201005_F@4@D@441-2-1-20-Z18_D@441-2-1-20-Z18_D-0000021.jpg 로드 완료
Day08_201005_F@4@D@441-2-1-20-Z18_D@441-2-1-20-Z18_D-0000023.jpg 로드 완료
Day08_201005_F@4@D@441-2-1-20-Z18_D@441-2-1-20-Z18_D-0000025.jpg 로드 완료
Day08_201005_F@4@D@441-2-1-20-Z18_D@441-2-1-20-Z18_D-0000027.jpg 로드 완료
Day08_

Day22_201023_F@6@B@793-3-5-41-Z112_B@793-3-5-41-Z112_B-0000027.jpg 로드 완료
Day28_201030_F@9@D@221-2-1-12-Z87_D@221-2-1-12-Z87_D-0000003.jpg 로드 완료
Day22_201023_F@7@D@708-3-5-34-Z113_D@708-3-5-34-Z113_D-0000013.jpg 로드 완료
Day40_201113_F@12@E@686-2-4-33-Z84_E@686-2-4-33-Z84_E-0000009.jpg 로드 완료
Day30_201102_F@6@A@036-1-1-21-Z111_A@036-1-1-21-Z111_A-0000031.jpg 로드 완료
Day25_201027_F@9@B@313-2-1-15-Z76_B@313-2-1-15-Z76_B-0000001.jpg 로드 완료
Day09_201006_F@7@C@104-1-1-04-Z38_C@104-1-1-04-Z38_C-0000019.jpg 로드 완료
Day21_201022_F@3@A@784-3-5-40-Z101_A@784-3-5-40-Z101_A-0000027.jpg 로드 완료
Day17_201017_F@2@E@775-3-5-40-Z72_E@775-3-5-40-Z72_E-0000003.jpg 로드 완료
Day17_201017_F@6@E@722-3-5-35-Z4_E@722-3-5-35-Z4_E-0000025.jpg 로드 완료
Day13_201013_F@8@A@083-1-1-04-Z58_A@083-1-1-04-Z58_A-0000003.jpg 로드 완료
Day39_201112_F@12@C@685-2-4-33-Z7_C@685-2-4-33-Z7_C-0000015.jpg 로드 완료
Day17_201017_F@5@D@789-3-5-41-Z74_D@789-3-5-41-Z74_D-0000003.jpg 로드 완료
Day14_201014_F@1@E@203-2-1-10-Z51_E@203-2-1-10-Z51_E-0000027.jpg 로드 완료


Day33_201105_F@2@E@516-1-2-24-Z107_E@516-1-2-24-Z107_E-0000007.jpg 로드 완료
Day05_200925_F@2@B@026-1-1-01-Z18_B@026-1-1-01-Z18_B-0000003.jpg 로드 완료
Day22_201023_F@7@D@752-3-5-37-Z113_D@752-3-5-37-Z113_D-0000025.jpg 로드 완료
Day11_201008_F@3@A@335-2-1-16-Z35_A@335-2-1-16-Z35_A-0000025.jpg 로드 완료
Day39_201112_F@12@B@687-2-4-33-Z7_B@687-2-4-33-Z7_B-0000003.jpg 로드 완료
Day22_201023_F@4@C@706-3-5-34-Z110_C@706-3-5-34-Z110_C-0000021.jpg 로드 완료
Day09_201006_F@4@D@027-1-1-01-Z36_D@027-1-1-01-Z36_D-0000017.jpg 로드 완료
Day11_201008_F@5@D@253-2-1-13-Z34_D@253-2-1-13-Z34_D-0000011.jpg 로드 완료
Day23_201024_F@3@E@068-1-1-03-Z101_E@068-1-1-03-Z101_E-0000031.jpg 로드 완료
Day42_201116_F@1@B@679-2-4-33-Z155_B@679-2-4-33-Z155_B-0000003.jpg 로드 완료
Day39_201112_F@14@A@644-2-4-31-Z106_A@644-2-4-31-Z106_A-0000009.jpg 로드 완료
Day20_201021_F@9@B@705-3-5-34-Z99_B@705-3-5-34-Z99_B-0000003.jpg 로드 완료
Day17_201017_F@6@E@740-3-5-37-Z4_E@740-3-5-37-Z4_E-0000031.jpg 로드 완료
Day22_201023_F@2@E@783-3-5-40-Z108_E@783-3-5-40-Z108_E-0000023.jpg 

Day34_201106_F@1@E@591-1-3-27-Z37_E@591-1-3-27-Z37_E-0000005.jpg 로드 완료
Day17_201017_F@2@C@700-3-5-34-Z72_C@700-3-5-34-Z72_C-0000013.jpg 로드 완료
Day41_201114_F@15@C@664-2-4-32-Z72_C@664-2-4-32-Z72_C-0000017.jpg 로드 완료
Day05_200925_F@2@B@055-1-1-03-Z18_B@055-1-1-03-Z18_B-0000027.jpg 로드 완료
Day10_201007_F@4@E@176-1-1-06-Z37_E@176-1-1-06-Z37_E-0000017.jpg 로드 완료
Day17_201017_F@1@C@783-3-5-40-Z39_C@783-3-5-40-Z39_C-0000029.jpg 로드 완료
Day05_200925_F@6@C@099-1-1-04-Z22_C@099-1-1-04-Z22_C-0000011.jpg 로드 완료
Day09_201006_F@3@C@018-1-1-01-Z35_C@018-1-1-01-Z35_C-0000015.jpg 로드 완료
Day10_201007_F@5@A@193-2-1-09-Z33_A@193-2-1-09-Z33_A-0000015.jpg 로드 완료
Day11_201008_F@6@A@259-2-1-13-Z40_A@259-2-1-13-Z40_A-0000031.jpg 로드 완료
Day22_201023_F@8@B@753-3-5-38-Z114_B@753-3-5-38-Z114_B-0000013.jpg 로드 완료
Day28_201030_F@2@B@212-2-1-11-Z107_B@212-2-1-11-Z107_B-0000001.jpg 로드 완료
Day03_200923_F@6@A@344-2-1-16-Z5_A@344-2-1-16-Z5_A-0000017.jpg 로드 완료
Day39_201112_F@13@B@689-2-4-30-Z75_B@689-2-4-30-Z75_B-0000015.jpg 로드 완료
Da

Day21_201022_F@9@B@753-3-5-38-Z107_B@753-3-5-38-Z107_B-0000013.jpg 로드 완료
Day20_201021_F@6@B@773-3-5-40-Z96_B@773-3-5-40-Z96_B-0000003.jpg 로드 완료
Day10_201007_F@3@B@190-1-1-08-Z35_B@190-1-1-08-Z35_B-0000009.jpg 로드 완료
Day21_201022_F@3@A@788-3-5-41-Z101_A@788-3-5-41-Z101_A-0000017.jpg 로드 완료
Day11_201008_F@7@C@284-2-1-14-Z38_C@284-2-1-14-Z38_C-0000027.jpg 로드 완료
Day11_201008_F@4@D@276-2-1-13-Z37_D@276-2-1-13-Z37_D-0000027.jpg 로드 완료
Day17_201017_F@9@A@758-3-5-38-Z75_A@758-3-5-38-Z75_A-0000021.jpg 로드 완료
Day22_201023_F@9@A@739-3-5-37-Z4_A@739-3-5-37-Z4_A-0000003.jpg 로드 완료
Day11_201008_F@7@A@297-2-1-14-Z38_A@297-2-1-14-Z38_A-0000003.jpg 로드 완료
Day41_201114_F@16@A@661-2-4-32-Z154_A@661-2-4-32-Z154_A-0000019.jpg 로드 완료
Day15_201015_F@8@A@343-2-1-16-Z58_A@343-2-1-16-Z58_A-0000031.jpg 로드 완료
Day22_201023_F@7@E@706-3-5-34-Z113_E@706-3-5-34-Z113_E-0000031.jpg 로드 완료
Day17_201017_F@8@C@697-3-5-34-Z8_C@697-3-5-34-Z8_C-0000017.jpg 로드 완료
Day27_201029_F@5@C@115-1-1-05-Z62_C@115-1-1-05-Z62_C-0000017.jpg 로드 완료
D

...

Day08_201005_F@6@A@463-2-1-20-Z21_A@463-2-1-20-Z21_A-0000023.jpg 로드 완료
Day23_201024_F@1@D@043-1-1-02-Z72_D@043-1-1-02-Z72_D-0000027.jpg 로드 완료
Day10_201007_F@8@A@166-1-1-06-Z39_A@166-1-1-06-Z39_A-0000015.jpg 로드 완료
Day20_201021_F@2@B@768-3-5-39-Z92_B@768-3-5-39-Z92_B-0000005.jpg 로드 완료
Day22_201023_F@8@D@745-3-5-37-Z114_D@745-3-5-37-Z114_D-0000005.jpg 로드 완료
Day24_201026_F@6@D@142-1-1-05-Z104_D@142-1-1-05-Z104_D-0000025.jpg 로드 완료
Day17_201017_F@2@C@728-3-5-35-Z72_C@728-3-5-35-Z72_C-0000029.jpg 로드 완료
Day17_201017_F@9@C@769-3-5-40-Z75_C@769-3-5-40-Z75_C-0000013.jpg 로드 완료
Day34_201106_F@2@A@555-1-2-26-Z54_A@555-1-2-26-Z54_A-0000025.jpg 로드 완료
Day11_201008_F@9@E@329-2-1-16-Z41_E@329-2-1-16-Z41_E-0000015.jpg 로드 완료
Day17_201017_F@9@B@705-3-5-34-Z75_B@705-3-5-34-Z75_B-0000009.jpg 로드 완료
Day28_201030_F@5@E@148-1-1-06-Z108_E@148-1-1-06-Z108_E-0000001.jpg 로드 완료
Day11_201008_F@3@B@252-2-1-13-Z35_B@252-2-1-13-Z35_B-0000025.jpg 로드 완료
Day30_201102_F@8@E@438-2-1-19-Z114_E@438-2-1-19-Z114_E-0000027.jpg 로드 완

Day42_201116_F@7@C@644-2-4-31-Z161_C@644-2-4-31-Z161_C-0000025.jpg 로드 완료
Day21_201022_F@6@C@711-3-5-34-Z104_C@711-3-5-34-Z104_C-0000011.jpg 로드 완료
Day20_201021_F@3@C@699-3-5-34-Z93_C@699-3-5-34-Z93_C-0000003.jpg 로드 완료
Day21_201022_F@4@D@791-3-5-41-Z102_D@791-3-5-41-Z102_D-0000027.jpg 로드 완료
Day34_201106_F@3@E@473-1-2-21-Z98_E@473-1-2-21-Z98_E-0000025.jpg 로드 완료
Day13_201013_F@6@D@057-1-1-03-Z56_D@057-1-1-03-Z56_D-0000031.jpg 로드 완료
Day09_201006_F@4@C@096-1-1-04-Z36_C@096-1-1-04-Z36_C-0000025.jpg 로드 완료
Day42_201116_F@6@A@690-2-4-33-Z160_A@690-2-4-33-Z160_A-0000023.jpg 로드 완료
Day17_201017_F@1@C@808-3-5-41-Z39_C@808-3-5-41-Z39_C-0000013.jpg 로드 완료
Day23_201024_F@6@E@017-1-1-01-Z104_E@017-1-1-01-Z104_E-0000019.jpg 로드 완료
Day29_201031_F@4@C@350-2-1-16-Z113_C@350-2-1-16-Z113_C-0000013.jpg 로드 완료
Day39_201112_F@5@A@652-2-4-32-Z141_A@652-2-4-32-Z141_A-0000021.jpg 로드 완료
Day21_201022_F@2@E@794-3-5-41-Z100_E@794-3-5-41-Z100_E-0000023.jpg 로드 완료
Day23_201024_F@6@B@023-1-1-01-Z104_B@023-1-1-01-Z104_B-000002

Day11_201008_F@3@A@291-2-1-14-Z35_A@291-2-1-14-Z35_A-0000031.jpg 로드 완료
Day15_201015_F@3@A@282-2-1-14-Z57_A@282-2-1-14-Z57_A-0000001.jpg 로드 완료
Day17_201017_F@5@E@803-3-5-41-Z74_E@803-3-5-41-Z74_E-0000009.jpg 로드 완료
Day01_200921_F@2@D@100-1-1-04-Z2_D@100-1-1-04-Z2_D-0000005.jpg 로드 완료
Day10_201007_F@9@D@200-2-1-09-Z41_D@200-2-1-09-Z41_D-0000017.jpg 로드 완료
Day22_201023_F@7@E@780-3-5-40-Z113_E@780-3-5-40-Z113_E-0000011.jpg 로드 완료
Day04_200924_F@6@C@372-2-1-17-Z8_C@372-2-1-17-Z8_C-0000007.jpg 로드 완료
Day13_201013_F@1@A@012-1-1-01-Z51_A@012-1-1-01-Z51_A-0000015.jpg 로드 완료
Day36_201108_F@2@B@542-1-2-24-Z136_B@542-1-2-24-Z136_B-0000023.jpg 로드 완료
Day10_201007_F@9@C@154-1-1-06-Z41_C@154-1-1-06-Z41_C-0000007.jpg 로드 완료
Day05_200925_F@6@E@065-1-1-03-Z22_E@065-1-1-03-Z22_E-0000023.jpg 로드 완료
Day21_201022_F@4@D@765-3-5-39-Z102_D@765-3-5-39-Z102_D-0000007.jpg 로드 완료
Day22_201023_F@7@A@750-3-5-37-Z113_A@750-3-5-37-Z113_A-0000031.jpg 로드 완료
Day20_201021_F@8@A@785-3-5-41-Z98_A@785-3-5-41-Z98_A-0000007.jpg 로드 완료
Da

Day05_200925_F@6@D@110-1-1-04-Z22_D@110-1-1-04-Z22_D-0000009.jpg 로드 완료
Day26_201028_F@5@D@434-2-1-19-Z101_D@434-2-1-19-Z101_D-0000003.jpg 로드 완료
Day21_201022_F@3@E@709-3-5-34-Z101_E@709-3-5-34-Z101_E-0000031.jpg 로드 완료
Day22_201023_F@1@E@754-3-5-38-Z108_E@754-3-5-38-Z108_E-0000023.jpg 로드 완료
Day05_200925_F@8@B@095-1-1-04-Z24_B@095-1-1-04-Z24_B-0000029.jpg 로드 완료
Day17_201017_F@1@D@791-3-5-41-Z39_D@791-3-5-41-Z39_D-0000027.jpg 로드 완료
Day17_201017_F@8@D@732-3-5-36-Z8_D@732-3-5-36-Z8_D-0000025.jpg 로드 완료
Day04_200924_F@4@A@391-2-1-18-Z1_A@391-2-1-18-Z1_A-0000003.jpg 로드 완료
Day02_200922_F@5@C@183-1-1-07-Z3_C@183-1-1-07-Z3_C-0000003.jpg 로드 완료
Day30_201102_F@7@C@436-2-1-19-Z112_C@436-2-1-19-Z112_C-0000019.jpg 로드 완료
Day20_201021_F@2@D@790-3-5-41-Z92_D@790-3-5-41-Z92_D-0000015.jpg 로드 완료
Day02_200922_F@7@D@213-2-1-11-Z7_D@213-2-1-11-Z7_D-0000027.jpg 로드 완료
Day10_201007_F@2@B@236-2-1-12-Z36_B@236-2-1-12-Z36_B-0000023.jpg 로드 완료
Day20_201021_F@5@D@760-3-5-38-Z95_D@760-3-5-38-Z95_D-0000027.jpg 로드 완료
Day22_

Day20_201021_F@2@B@797-3-5-41-Z92_B@797-3-5-41-Z92_B-0000021.jpg 로드 완료
Day27_201029_F@3@C@031-1-1-01-Z113_C@031-1-1-01-Z113_C-0000027.jpg 로드 완료
Day15_201015_F@7@A@339-2-1-16-Z53_A@339-2-1-16-Z53_A-0000019.jpg 로드 완료
Day22_201023_F@3@A@765-3-5-39-Z110_A@765-3-5-39-Z110_A-0000031.jpg 로드 완료
Day34_201106_F@4@D@494-1-2-22-Z131_D@494-1-2-22-Z131_D-0000031.jpg 로드 완료
Day21_201022_F@6@C@802-3-5-41-Z104_C@802-3-5-41-Z104_C-0000003.jpg 로드 완료
Day20_201021_F@6@B@735-3-5-36-Z96_B@735-3-5-36-Z96_B-0000007.jpg 로드 완료
Day26_201028_F@1@E@442-2-1-20-Z99_E@442-2-1-20-Z99_E-0000005.jpg 로드 완료
Day33_201105_F@2@B@581-1-3-27-Z107_B@581-1-3-27-Z107_B-0000029.jpg 로드 완료
Day04_200924_F@7@A@362-2-1-17-Z3_A@362-2-1-17-Z3_A-0000011.jpg 로드 완료
Day21_201022_F@9@D@760-3-5-38-Z107_D@760-3-5-38-Z107_D-0000011.jpg 로드 완료
Day22_201023_F@3@D@786-3-5-41-Z110_D@786-3-5-41-Z110_D-0000003.jpg 로드 완료
Day22_201023_F@9@A@804-3-5-41-Z4_A@804-3-5-41-Z4_A-0000013.jpg 로드 완료
Day21_201022_F@7@E@735-3-5-36-Z105_E@735-3-5-36-Z105_E-0000025.jpg 

In [30]:
train_X.shape

(30, 16, 224, 224, 3)

In [32]:
train_X

array([[[[[0.99607843, 0.99607843, 0.99607843],
          [0.99607843, 0.99607843, 0.99607843],
          [0.99607843, 0.99607843, 0.99607843],
          ...,
          [0.98039216, 0.98039216, 0.98039216],
          [0.98039216, 0.98039216, 0.98039216],
          [0.98039216, 0.98039216, 0.98039216]],

         [[0.99607843, 0.99607843, 0.99607843],
          [0.99607843, 0.99607843, 0.99607843],
          [0.99607843, 0.99607843, 0.99607843],
          ...,
          [0.98039216, 0.98039216, 0.98039216],
          [0.98039216, 0.98039216, 0.98039216],
          [0.98039216, 0.98039216, 0.98039216]],

         [[0.99607843, 0.99607843, 0.99607843],
          [0.99607843, 0.99607843, 0.99607843],
          [0.99607843, 0.99607843, 0.99607843],
          ...,
          [0.98039216, 0.98039216, 0.98039216],
          [0.98039216, 0.98039216, 0.98039216],
          [0.98039216, 0.98039216, 0.98039216]],

         ...,

         [[0.63529414, 0.63529414, 0.63529414],
          [0.6392157 ,

In [ ]:
train_X = np.array(train_X, dtype=np.float32) / 255.0
train_X

In [ ]:
test_data.drop(index=faild_idx, inplace=True)

In [ ]:
test_data.info()

In [ ]:
test_data.type.to_list()

In [40]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# 라벨 인코딩 (문자형 → 숫자형)
label_encoder = LabelEncoder()
train_y_encoded = label_encoder.fit_transform(test_data.type.to_list())

# 원-핫 인코딩 (숫자형 → 원-핫 벡터)
train_y_onehot = to_categorical(train_y_encoded)

# 변환된 결과 확인
print("라벨 인코딩된 train_y:", train_y_encoded[:10])  # [0, 1, 2, ...] 형태
print("원-핫 인코딩된 train_y:\n", train_y_onehot[:10])  # [[1, 0,


라벨 인코딩된 train_y: [0 2 0 1 2 0 2 1 0 2]
원-핫 인코딩된 train_y:
 [[1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [41]:
import tensorflow as tf
from tensorflow.keras.applications import NASNetMobile
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# NASNet 모델 불러오기 (사전 학습된 가중치 사용)
base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 사전 학습된 층을 동결 (Feature Extractor로 사용)
base_model.trainable = False  

# NASNet 위에 커스텀 레이어 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)  # GAP 레이어 추가 (평균 풀링)
x = Dropout(0.5)(x)  # 과적합 방지를 위한 Dropout
x = Dense(512, activation='relu')(x)  # Fully Connected Layer
x = Dropout(0.3)(x)  
predictions = Dense(5, activation='softmax')(x)  # 운동 클래스 개수에 맞게 수정

# 최종 모델 구성
model = Model(inputs=base_model.input, outputs=predictions)

# 모델 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

model.summary()  # 모델 구조 확인


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv1 (Conv2D)            (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn1 (BatchNormalization)  (None, 111, 111, 32  128         ['stem_conv1[0][0]']             
                                )                                                             

In [42]:
from tensorflow.keras.applications import ResNet50, MobileNetV2, EfficientNetB7, NASNetMobile
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten, Dropout

# 사용할 모델 선택
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))  # ResNet50
# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(256, 256, 3))  # MobileNetV2
# base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(256, 256, 3))  # EfficientNetB7
base_model = NASNetMobile(weights='imagenet', include_top=False, input_shape=(256,256,3))
base_model.trainable = False

# 추가 레이어 정의
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)  # Fully Connected Layer 추가 가능
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output_layer = Dense(3, activation='softmax')(x)  # 클래스 개수에 맞게 설정

# 새 모델 생성
model = Model(inputs=base_model.input, outputs=output_layer)

# 모델 구조 확인
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv1 (Conv2D)            (None, 127, 127, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 stem_bn1 (BatchNormalization)  (None, 127, 127, 32  128         ['stem_conv1[0][0]']             
                                )                                                           

In [43]:
model.compile(
    optimizer='adam',  # Adam 옵티마이저 사용
    loss='categorical_crossentropy',  # 다중 분류 문제
    metrics=['accuracy']  # 정확도 측정
)

In [44]:
hist = model.fit(train_X,train_y_onehot, validation_split=0.3, epochs=100, batch_size=4)

Epoch 1/100
359/359 [==============================] - 28s 53ms/step - loss: 1.1794 - accuracy: 0.3489 - val_loss: 1.1091 - val_accuracy: 0.3295
Epoch 2/100
359/359 [==============================] - 17s 46ms/step - loss: 1.1219 - accuracy: 0.3231 - val_loss: 1.1099 - val_accuracy: 0.3490
Epoch 3/100
359/359 [==============================] - 17s 47ms/step - loss: 1.1056 - accuracy: 0.3370 - val_loss: 1.0994 - val_accuracy: 0.3328
Epoch 4/100
359/359 [==============================] - 17s 48ms/step - loss: 1.1027 - accuracy: 0.3475 - val_loss: 1.0980 - val_accuracy: 0.3506
Epoch 5/100
359/359 [==============================] - 17s 48ms/step - loss: 1.1024 - accuracy: 0.3558 - val_loss: 1.1025 - val_accuracy: 0.3198
Epoch 6/100
359/359 [==============================] - 17s 48ms/step - loss: 1.0979 - accuracy: 0.3600 - val_loss: 1.1031 - val_accuracy: 0.3198
Epoch 7/100
359/359 [==============================] - 17s 48ms/step - loss: 1.0936 - accuracy: 0.3614 - val_loss: 1.1191 - val_ac

In [ ]:
y_hat = model.predict(train_X)

In [ ]:
y_hat = np.argmax(y_hat, axis=1)

In [ ]:
real_data = np.argmax(train_y_onehot, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
accuracy = accuracy_score(real_data, y_hat)

In [ ]:
train_y.shape

In [ ]:
accuracy